# Modeling

In [40]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from keras.optimizers import Adam
import keras
import pickle

In [2]:
#os.chdir("C:\\Users\\aksha\\OneDrive - The University of Chicago\\Files\\Courses\\Machine_Learning_and_PredictiveAnalytics\\Project\\ML_Project\\")
data = pd.read_csv('data2.csv')

In [3]:
data.head()

,Unnamed: 0,rev_id,comment,year,logged_in,ns,sample,split,toxic_label_mode,toxic_score_mean
0,0,2232.0,This: :One can make an analogy in mathematical...,2002,True,article,random,train,0,0.4
1,10,4216.0,` :Clarification for you (and Zundark's righ...,2002,True,user,random,train,0,0.5
2,20,8953.0,Elected or Electoral? JHK,2002,False,article,random,test,0,0.1
3,30,26547.0,`This is such a fun entry. Devotchka I once...,2002,True,article,random,train,0,0.6
4,40,28959.0,Please relate the ozone hole to increases in c...,2002,True,article,random,test,0,0.2


In [4]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data['toxic_label_mode'])

In [5]:
print(data['toxic_label_mode'].unique())

['0' '1' '[0 1]']


In [6]:
data['toxic_label_mode'].value_counts()

0        111930
1         12483
[0 1]      2442
Name: toxic_label_mode, dtype: int64

In [8]:
xtrain= data.loc[data.split=="train",["comment"]]
xvalid= data.loc[data.split=="dev",["comment"]]
ytrain= data.loc[data.split=="train",["toxic_label_mode"]]
yvalid= data.loc[data.split=="dev",["toxic_label_mode"]]

In [9]:
print (xtrain.shape)
print (xvalid.shape)

(76032, 1)
(25442, 1)


## Word Representations

### TFIDF 

In [24]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [25]:
tfv.fit(list(xtrain.comment) + list(xvalid.comment))
xtrain_tfv =  tfv.transform(xtrain.comment) 
xvalid_tfv = tfv.transform(xvalid.comment)

In [41]:
filename = 'tfv.pkl'
pickle.dump(tfv, open(filename, 'wb'))

### Count Vectorizer

In [15]:
#using countvectorizer
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain.comment) + list(xvalid.comment))
xtrain_ctv =  ctv.transform(xtrain.comment) 
xvalid_ctv = ctv.transform(xvalid.comment)

### GLOVE

In [10]:
import os
# load the GloVe vectors in a dictionary:
#os.chdir("C:\\Users\\SIDDARTH\\Desktop\\CUTE 5")
embeddings_index = {}
failed_words= []
f = open('glove.840B.300d.txt',encoding='utf-8')
for line in tqdm(f):
    try:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except:
        values = line.split()
        word = values[0]     
        failed_words.append(word)
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196018it [03:05, 11841.30it/s]

Found 2195885 word vectors.


In [11]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [12]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain.comment)]


100%|██████████| 76032/76032 [00:50<00:00, 1502.99it/s]


In [13]:
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid.comment)]

100%|██████████| 25442/25442 [00:17<00:00, 1422.60it/s]


In [14]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [15]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

## Logistic Model

### Logistic + TFIDF

In [57]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_tfv, yvalid)))

C:\Users\aksha\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.9377014385661505


C:\Users\aksha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [58]:
filename = 'Logistic_model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [48]:
import pickle
filename = 'Logistic_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [14]:
import pickle
filename = 'Logistic_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtrain_tfv, ytrain)
print(result)

0.9493765782828283


C:\Users\aksha\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


### Logistic + Count Vector

In [51]:
clf = LogisticRegression(penalty='l2',C=1.0)
clf.fit(xtrain_ctv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_ctv, yvalid)))
filename = 'Logistic_model_countvectorizer.sav'
pickle.dump(clf, open(filename, 'wb'))

C:\Users\SIDDARTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\SIDDARTH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9390378114928072


In [16]:
import pickle
filename = 'Logistic_model_countvectorizer.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtrain_ctv, ytrain)
print(result)

C:\Users\aksha\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0.9978430134680135


### Logistic + Glove

In [18]:
clf = LogisticRegression(penalty='l2',C=1.0)
clf.fit(xtrain_glove_scl, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_glove_scl, yvalid)))
filename = 'Logistic_model_glove.pkl'
pickle.dump(clf, open(filename, 'wb'))

C:\Users\aksha\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.9389198962345727


C:\Users\aksha\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
# Train Accuracy
filename = 'Logistic_model_glove.pkl'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtrain_glove_scl, ytrain)
print(result)

0.9394333964646465


## Naive Bayes

### Naive Bayes + TFIDF

In [22]:
#Fitting Naive bayes
clf = MultinomialNB(alpha=0.1)
clf.fit(xtrain_tfv, ytrain)
print("Accuracy: {}".format(clf.score(xvalid_tfv, yvalid)))
filename = 'NB_TFIDF.sav'
pickle.dump(clf, open(filename, 'wb'))

C:\Users\aksha\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Accuracy: 0.9276000314440689


In [23]:
filename = 'NB_TFIDF.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtrain_tfv, ytrain)
print(result)

0.9592276936026936


### Naive Bayes + Count Vector

In [53]:
clf=MultinomialNB(alpha=1)
clf.fit(xtrain_ctv,ytrain)
predictions = clf.predict_proba(xvalid_ctv)
print("Accuracy: {}".format(clf.score(xvalid_ctv, yvalid)))
filename = 'NB_Countvectorizer.sav'
pickle.dump(clf, open(filename, 'wb'))

C:\Users\SIDDARTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Accuracy: 0.917970285354925


In [21]:
filename = 'NB_Countvectorizer.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(xtrain_ctv, ytrain)
print(result)

C:\Users\aksha\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0.975641835016835


In [8]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, 
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_ctv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  1.4min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  1.4min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  1.4min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  1.4min remaining:   28.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.4min finished
C:\Users\SIDDARTH\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:847: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  warnings.warn(
C:\Users\SIDDARTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Best score: 0.911
Best parameters set:
	nb__alpha: 1


### Multi Layer Perceptron

In [32]:
# Replacing the neutral comments [0,1] to non toxic = 0
data['toxic_label_mode'] = np.where(data['toxic_label_mode'] == '[0 1]', '0', data['toxic_label_mode'])

In [33]:
ytrain= data.loc[data.split=="train",["toxic_label_mode"]]
yvalid= data.loc[data.split=="dev",["toxic_label_mode"]]

In [34]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

In [35]:
import keras.backend as K 
def recall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = TP / (PP + K.epsilon())
    return recall

In [43]:
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(2))
model.add(Activation('softmax'))


# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', keras.metrics.Recall()])

In [44]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=50, verbose=2, 
          validation_data=(xvalid_glove_scl, yvalid_enc),callbacks=[earlystop])

Epoch 1/50
1188/1188 - 6s - loss: 0.1923 - accuracy: 0.9281 - recall_1: 0.9281 - val_loss: 0.1276 - val_accuracy: 0.9537 - val_recall_1: 0.9537
Epoch 2/50
1188/1188 - 4s - loss: 0.1349 - accuracy: 0.9499 - recall_1: 0.9499 - val_loss: 0.1237 - val_accuracy: 0.9556 - val_recall_1: 0.9556
Epoch 3/50
1188/1188 - 4s - loss: 0.1269 - accuracy: 0.9522 - recall_1: 0.9522 - val_loss: 0.1234 - val_accuracy: 0.9552 - val_recall_1: 0.9552
Epoch 4/50
1188/1188 - 4s - loss: 0.1235 - accuracy: 0.9534 - recall_1: 0.9534 - val_loss: 0.1238 - val_accuracy: 0.9544 - val_recall_1: 0.9544
Epoch 5/50
1188/1188 - 4s - loss: 0.1172 - accuracy: 0.9564 - recall_1: 0.9564 - val_loss: 0.1245 - val_accuracy: 0.9552 - val_recall_1: 0.9552
Epoch 6/50
1188/1188 - 4s - loss: 0.1125 - accuracy: 0.9584 - recall_1: 0.9584 - val_loss: 0.1210 - val_accuracy: 0.9559 - val_recall_1: 0.9559
Epoch 7/50
1188/1188 - 4s - loss: 0.1069 - accuracy: 0.9593 - recall_1: 0.9593 - val_loss: 0.1254 - val_accuracy: 0.9556 - val_recall_1:

In [45]:
# serialize model to JSON
model_json = model.to_json()
with open("MLP_embeddings.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("MLP_embeddings.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
## load json and create model
#json_file = open('MLP_embeddings.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
## load weights into new model
#loaded_model.load_weights("MLP_embeddings.h5")
#print("Loaded model from disk")

## LSTM + Glove

In [46]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid.comment))
xtrain_seq = token.texts_to_sequences(xtrain.comment)
xvalid_seq = token.texts_to_sequences(xvalid.comment)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [47]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 66695/66695 [00:00<00:00, 424723.60it/s]


In [50]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300,dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(2))
model.add(Activation('softmax'))
from keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy',recall])

In [51]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=3, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=256, epochs=10, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/10
297/297 [==============================] - 554s 2s/step - loss: 0.2299 - accuracy: 0.9178 - recall: 0.9178 - val_loss: 0.1148 - val_accuracy: 0.9577 - val_recall: 0.9579
Epoch 2/10
297/297 [==============================] - 621s 2s/step - loss: 0.1342 - accuracy: 0.9519 - recall: 0.9519 - val_loss: 0.1110 - val_accuracy: 0.9600 - val_recall: 0.9601
Epoch 3/10
297/297 [==============================] - 651s 2s/step - loss: 0.1263 - accuracy: 0.9575 - recall: 0.9575 - val_loss: 0.1177 - val_accuracy: 0.9566 - val_recall: 0.9566
Epoch 4/10
297/297 [==============================] - 915s 3s/step - loss: 0.1167 - accuracy: 0.9583 - recall: 0.9583 - val_loss: 0.1062 - val_accuracy: 0.9621 - val_recall: 0.9622
Epoch 5/10
297/297 [==============================] - 805s 3s/step - loss: 0.1185 - accuracy: 0.9577 - recall: 0.9577 - val_loss: 0.1131 - val_accuracy: 0.9624 - val_recall: 0.9625
Epoch 6/10
297/297 [==============================] - 728s 2s/step - loss: 0.1151 - accuracy: 0

In [52]:
# serialize model to JSON
model_json = model.to_json()
with open("LSTM_embeddings.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("LSTM_embeddings.h5")
print("Saved model to disk")

Saved model to disk


## GRU Model + Glove 

In [53]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=3, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=20, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/20
149/149 [==============================] - 1273s 9s/step - loss: 0.2408 - accuracy: 0.9091 - val_loss: 0.1145 - val_accuracy: 0.9587
Epoch 2/20
149/149 [==============================] - 1504s 10s/step - loss: 0.1300 - accuracy: 0.9535 - val_loss: 0.1089 - val_accuracy: 0.9607
Epoch 3/20
149/149 [==============================] - 1681s 11s/step - loss: 0.1210 - accuracy: 0.9553 - val_loss: 0.1049 - val_accuracy: 0.9613
Epoch 4/20
149/149 [==============================] - 1638s 11s/step - loss: 0.1144 - accuracy: 0.9580 - val_loss: 0.1036 - val_accuracy: 0.9621
Epoch 5/20
149/149 [==============================] - 1655s 11s/step - loss: 0.1120 - accuracy: 0.9589 - val_loss: 0.1049 - val_accuracy: 0.9616
Epoch 6/20
149/149 [==============================] - 1606s 11s/step - loss: 0.1088 - accuracy: 0.9601 - val_loss: 0.1049 - val_accuracy: 0.9623
Epoch 7/20
149/149 [==============================] - 1538s 10s/step - loss: 0.1079 - accuracy: 0.9610 - val_loss: 0.1003 - val_acc

In [54]:
# serialize model to JSON
model_json = model.to_json()
with open("GRU_embeddings.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("GRU_embeddings.h5")
print("Saved model to disk")

Saved model to disk
